In [1]:
# Import Pandas
import pandas as pd

# Import preprocessing methods and train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Import Counter
from collections import Counter

# Import Sampling Methods
from imblearn.over_sampling import RandomOverSampler

# Import Machine Learning Models
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier
import tensorflow as tf

# Import Methods for Metric Reporting
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

## Inspecting the Data Before Preprocessing

In [2]:
# Load and inspect the merged_spotify_songs.csv dataset
spotify_df = pd.read_csv("../Resources/merged_spotify_songs.csv")
spotify_df.head(10)

,id,name,artists,release_date,year,duration_ms,acousticness,danceability,energy,explicit,...,key,liveness,loudness,loudness_scaled,mode,popularity,speechiness,tempo,tempo_scaled,valence
0,02GDntOXexBFUvSgaXLPkd,Morceaux de fantaisie Op. 3: No. 2 Prélude in ...,['Sergei Rachmaninoff'],1921-01-01,1921,218773,0.993,0,0.088,0,...,1,0.3630,-21.091,0.609334,0,0.02,0.0456,92.867,0.380461,0.0731
1,08zfJvRLp7pjAb94MA9JmF,Il Etait Syndiqué,['Fortugé'],1921-01-01,1921,196560,0.982,1,0.257,0,...,8,0.5040,-16.415,0.682562,1,0.00,0.3990,109.378,0.448103,0.7710
2,0BMkRpQtDoKjcgzCpnqLNa,Dans La Vie Faut Pas S'en Faire,['Maurice Chevalier'],1921-01-01,1921,147133,0.995,0,0.260,0,...,9,0.2580,-16.894,0.675061,1,0.00,0.0557,85.146,0.348829,0.8260
3,0eQsdik7GTEy7M3UytCbSN,Morceaux de fantaisie Op. 3: No. 2 Prélude in ...,['Sergei Rachmaninoff'],1921-01-01,1921,218773,0.993,0,0.088,0,...,1,0.3630,-21.091,0.609334,0,0.00,0.0456,92.867,0.380461,0.0731
4,0H3k2CvJvHULnWChlbeFgx,La Vipère,['Georgel'],1921-01-01,1921,190800,0.990,0,0.363,0,...,5,0.2920,-12.562,0.742902,0,0.00,0.0546,174.532,0.715028,0.4930
5,0i7MdVu0tNEyUdgpCBilKe,Je M'donne,['Maurice Chevalier'],1921-01-01,1921,181733,0.996,1,0.274,0,...,2,0.3020,-14.001,0.720366,1,0.00,0.0410,79.218,0.324543,0.6780
6,0LcXzABeA84EgudqpNUN1I,Ud Taksimi,['Mehmet Kemiksiz'],1921-01-01,1921,184973,0.912,0,0.420,0,...,8,0.1080,-10.766,0.771028,0,0.00,0.1140,70.758,0.289884,0.2120
7,0NFeJgmTAV1kDfzSQNK41Z,10 Préludes Op. 23: No. 5 in G Minor. Alla marcia,['Sergei Rachmaninoff' 'Ruth Laredo'],1921-01-01,1921,221013,0.989,0,0.171,0,...,7,0.1160,-20.476,0.618965,0,0.03,0.0319,107.698,0.441221,0.2820
8,0Nk5f07H3JaEunGrYfbqHM,Come Back To Erin,['Phil Regan'],1921-01-01,1921,186467,0.957,0,0.212,0,...,2,0.2360,-13.300,0.731344,1,0.01,0.0358,85.726,0.351205,0.2180
9,0osXBirvQzPRfKSUDzHPCv,Korkma Sönmez,['Mehmet Kemiksiz'],1921-01-01,1921,155063,0.388,1,0.698,0,...,4,0.0768,-8.184,0.811463,0,0.00,0.0421,133.951,0.548775,0.7230


In [3]:
# Find the dimensions of the dataset
spotify_df.shape

(169797, 21)

In [4]:
# Generate descriptive statistics
spotify_df.describe()

,year,duration_ms,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness,loudness_scaled,mode,popularity,speechiness,tempo,tempo_scaled,valence
count,169797.000000,1.697970e+05,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000
mean,1977.221712,2.314078e+05,0.493309,0.601047,0.488541,0.084895,0.161962,5.200551,0.206696,-11.370476,0.761562,0.708570,0.315550,0.094066,116.946318,0.479110,0.532034
std,25.599875,1.213399e+05,0.376619,0.489684,0.267381,0.278727,0.309349,3.515181,0.176796,5.667187,0.088751,0.454423,0.215861,0.149961,30.728301,0.125889,0.262407
min,1921.000000,5.108000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1957.000000,1.710400e+05,0.094500,0.000000,0.263000,0.000000,0.000000,2.000000,0.098400,-14.470000,0.713022,0.000000,0.120000,0.034900,93.512000,0.383103,0.322000
50%,1978.000000,2.086000e+05,0.493000,1.000000,0.481000,0.000000,0.000204,5.000000,0.135000,-10.474000,0.775601,1.000000,0.330000,0.045000,114.773000,0.470206,0.544000
75%,1999.000000,2.629600e+05,0.888000,1.000000,0.710000,0.000000,0.086800,8.000000,0.263000,-7.118000,0.828158,1.000000,0.480000,0.075400,135.708000,0.555973,0.749000
max,2020.000000,5.403500e+06,0.996000,1.000000,1.000000,1.000000,1.000000,11.000000,1.000000,3.855000,1.000000,1.000000,1.000000,0.969000,244.091000,1.000000,1.000000


In [5]:
# Confirm data types
spotify_df.dtypes

id                   object
name                 object
artists              object
release_date         object
year                  int64
duration_ms           int64
acousticness        float64
danceability          int64
energy              float64
explicit              int64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
loudness_scaled     float64
mode                  int64
popularity          float64
speechiness         float64
tempo               float64
tempo_scaled        float64
valence             float64
dtype: object

In [6]:
# Find null values
for column in spotify_df.columns:
    print(f"Column {column} has {spotify_df[column].isnull().sum()} null values")

Column id has 0 null values
Column name has 0 null values
Column artists has 0 null values
Column release_date has 0 null values
Column year has 0 null values
Column duration_ms has 0 null values
Column acousticness has 0 null values
Column danceability has 0 null values
Column energy has 0 null values
Column explicit has 0 null values
Column instrumentalness has 0 null values
Column key has 0 null values
Column liveness has 0 null values
Column loudness has 0 null values
Column loudness_scaled has 0 null values
Column mode has 0 null values
Column popularity has 0 null values
Column speechiness has 0 null values
Column tempo has 0 null values
Column tempo_scaled has 0 null values
Column valence has 0 null values


In [7]:
# Check for duplicates
print(f"Duplicated entries: {spotify_df.duplicated().sum()}")

Duplicated entries: 0


In [8]:
# Create a new DataFrame that holds the string columns.
song_names_df = spotify_df[["id", "name", "artists", "release_date", "year", "duration_ms"]].copy()
song_names_df 

,id,name,artists,release_date,year,duration_ms
0,02GDntOXexBFUvSgaXLPkd,Morceaux de fantaisie Op. 3: No. 2 Prélude in ...,['Sergei Rachmaninoff'],1921-01-01,1921,218773
1,08zfJvRLp7pjAb94MA9JmF,Il Etait Syndiqué,['Fortugé'],1921-01-01,1921,196560
2,0BMkRpQtDoKjcgzCpnqLNa,Dans La Vie Faut Pas S'en Faire,['Maurice Chevalier'],1921-01-01,1921,147133
3,0eQsdik7GTEy7M3UytCbSN,Morceaux de fantaisie Op. 3: No. 2 Prélude in ...,['Sergei Rachmaninoff'],1921-01-01,1921,218773
4,0H3k2CvJvHULnWChlbeFgx,La Vipère,['Georgel'],1921-01-01,1921,190800
...,...,...,...,...,...,...
169792,3xuUW0IwM2VbVgZSbVORzR,"Ik Vaari Aa (From 'Raabta'"")""",['Arijit Singh'],2020-06-16,2020,274687
169793,4IJOll4wTwDrv0HxGLvWjb,Born This Way,['Lady Gaga'],2020-06-16,2020,261083
169794,2r6agJaA9OsWpQk6cIsy17,Everybody's Gotta Learn Sometime,['G-Eazy'],2020-06-17,2020,177657
169795,345aCD2ReEBsMqKE392DKM,Everybody's Gotta Learn Sometime,['G-Eazy'],2020-06-17,2020,177657


## Data Preprocessing for Machine Learning

In [9]:
# Drop non numberical and repeat columns from main DataFrame
spotify_df = spotify_df.drop(columns=["id", "name", "artists", "release_date", "year", "duration_ms", "loudness", "tempo"])
spotify_df.head()

,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness_scaled,mode,popularity,speechiness,tempo_scaled,valence
0,0.993,0,0.088,0,0.527,1,0.363,0.609334,0,0.02,0.0456,0.380461,0.0731
1,0.982,1,0.257,0,0.000,8,0.504,0.682562,1,0.00,0.3990,0.448103,0.7710
2,0.995,0,0.260,0,0.000,9,0.258,0.675061,1,0.00,0.0557,0.348829,0.8260
3,0.993,0,0.088,0,0.527,1,0.363,0.609334,0,0.00,0.0456,0.380461,0.0731
4,0.990,0,0.363,0,0.000,5,0.292,0.742902,0,0.00,0.0546,0.715028,0.4930


In [10]:
# key column represents the key the track is in:
# i.e 0 = C, 1 = C#/D♭, 2 = D, 3 = D#/E♭, ... , 11 = B
# Therefore, key column should be encoded

# Create OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded Dataframe
encode_df = pd.DataFrame(enc.fit_transform(spotify_df.key.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names_out(['key'])
encode_df.head(10)

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Merge the two DataFrames together and drop the key column
spotify_df = spotify_df.merge(encode_df,left_index=True,right_index=True).drop(columns=["key"])
spotify_df.head(10)

,acousticness,danceability,energy,explicit,instrumentalness,liveness,loudness_scaled,mode,popularity,speechiness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0.993,0,0.088,0,0.527000,0.3630,0.609334,0,0.02,0.0456,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.982,1,0.257,0,0.000000,0.5040,0.682562,1,0.00,0.3990,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.995,0,0.260,0,0.000000,0.2580,0.675061,1,0.00,0.0557,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.993,0,0.088,0,0.527000,0.3630,0.609334,0,0.00,0.0456,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.990,0,0.363,0,0.000000,0.2920,0.742902,0,0.00,0.0546,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.996,1,0.274,0,0.000000,0.3020,0.720366,1,0.00,0.0410,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.912,0,0.420,0,0.890000,0.1080,0.771028,0,0.00,0.1140,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,0.989,0,0.171,0,0.820000,0.1160,0.618965,0,0.03,0.0319,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.957,0,0.212,0,0.000222,0.2360,0.731344,1,0.01,0.0358,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.388,1,0.698,0,0.000002,0.0768,0.811463,0,0.00,0.0421,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Split the data into input (X) and output (y)

In [12]:
# Separate the dataset into features (X) and target (y)
y = spotify_df["danceability"]
X = spotify_df.drop(columns=["danceability"])

In [13]:
# Check the balance of our target values
y.value_counts()

1    102056
0     67741
Name: danceability, dtype: int64

## Split the Data into Training and Testing

In [14]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y, train_size=0.75)

## Scaling the data

In [15]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
X_train_scaled[0]

array([-1.15268942,  1.68613203, -0.30431929, -0.28803298, -0.73645423,
        0.88205072,  0.64275158,  0.71454759,  0.01504092,  0.24033872,
        0.50519174, -0.38005953, -0.28624297, -0.35388126, -0.21061483,
       -0.2872693 , -0.32468988, -0.23139342, -0.37359131, -0.25926669,
        2.94952072, -0.27682084, -0.2573815 ])

In [17]:
# Determine the shape of our training and testing sets.
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

(127347, 23)
(42450, 23)
(127347,)
(42450,)


## Machine Learning Models:
1. Logistic Regression
2. Decision Tree
3. Balanced Random Forest Classifier
4. Easy Ensemble AdaBoost Classifier
5. Deep Neural Network


## 1. Naive Random Oversampling + Logistic Regression

In [18]:
# Examine Data with Counter: Data needs to be resampled for Logistic Regression
Counter(y_train)

Counter({1: 76542, 0: 50805})

### Resample data with RandomOverSampler

In [19]:
# Resample the data with RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({1: 76542, 0: 76542})

### Define, Train, and Make Predictions for a Logistic Regression Model

In [20]:
# Define the logistic regression model
model = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

# Train the Logistic Regression model with the resampled data
model.fit(X_resampled, y_resampled)

# Create Make
y_pred = model.predict(X_test_scaled)

### Create the Confusion Matrix

In [21]:
# Create a Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

### Validate the Model

In [22]:
# Calculating the accuracy score.
acc_score = balanced_accuracy_score(y_test, y_pred)

In [23]:
# Displaying results
print("Model: Logistic Regression\n")
print("Confusion Matrix")
display(cm_df)
print(f"Balanced Accuracy Score: {acc_score}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Model: Logistic Regression

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,12898,4038
Actual 1,6177,19337


Balanced Accuracy Score: 0.759735302733198

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.76      0.72     16936
           1       0.83      0.76      0.79     25514

    accuracy                           0.76     42450
   macro avg       0.75      0.76      0.75     42450
weighted avg       0.77      0.76      0.76     42450



## 2. Decision Tree

### Define, Train, and Make Predictions for a Decision Tree Model

In [24]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier(random_state=1)

# Fitting the model.
model = model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
y_pred = model.predict(X_test_scaled)

### Create the Confusion Matrix

In [25]:
# Create a Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

### Validate the Model

In [26]:
# Calculating the accuracy score.
acc_score = balanced_accuracy_score(y_test, y_pred)

In [27]:
# Displaying results
print("Model: Decision Tree \n")
print("Confusion Matrix")
display(cm_df)
print(f"Balanced Accuracy Score: {acc_score}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Model: Decision Tree 

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,11485,5451
Actual 1,5397,20117


Balanced Accuracy Score: 0.7333051567009493

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.68      0.68     16936
           1       0.79      0.79      0.79     25514

    accuracy                           0.74     42450
   macro avg       0.73      0.73      0.73     42450
weighted avg       0.74      0.74      0.74     42450



##  3. Balanced Random Forest Classifier

### Define, Train, and Make Predictions for a Balanced Random Forest Classifier Model

In [28]:
# Define the Balanced Random Forest Classifier Model
rf_model = BalancedRandomForestClassifier(n_estimators=500, random_state=1)

# Resample the training data with BalancedRandomForestClassifier
rf_model.fit(X_train_scaled, y_train)

# Make prediction
y_pred = rf_model.predict(X_test_scaled)

### Create the Confusion Matrix

In [29]:
# Create a Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

### Validate the Model

In [30]:
# Calculating the accuracy score.
acc_score = balanced_accuracy_score(y_test, y_pred)

In [31]:
# Displaying results
print("Model: Balanced Random Forest Classifier\n")
print("Confusion Matrix")
display(cm_df)
print(f"Balanced Accuracy Score: {acc_score}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Model: Balanced Random Forest Classifier

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,13683,3253
Actual 1,4753,20761


Balanced Accuracy Score: 0.8108170344592829

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.81      0.77     16936
           1       0.86      0.81      0.84     25514

    accuracy                           0.81     42450
   macro avg       0.80      0.81      0.81     42450
weighted avg       0.82      0.81      0.81     42450



##  4. Easy Ensemble AdaBoost Classifier

### Define, Train, and Make Predictions for an Easy Ensemble AdaBoost Classifier Model

In [32]:
# Define the Balanced Random Forest Classifier Model
eec = EasyEnsembleClassifier(n_estimators=50, random_state=1)

# Resample the training data with BalancedRandomForestClassifier
eec.fit(X_train_scaled, y_train)

# Make prediction
y_pred = eec.predict(X_test_scaled)

### Create the Confusion Matrix

In [33]:
# Create a Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

### Validate the Model

In [34]:
# Calculating the accuracy score.
acc_score = balanced_accuracy_score(y_test, y_pred)

In [35]:
# Displaying results
print("Model: Easy Ensemble AdaBoost Classifier\n")
print("Confusion Matrix")
display(cm_df)
print(f"Balanced Accuracy Score: {acc_score}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Model: Easy Ensemble AdaBoost Classifier

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,13240,3696
Actual 1,5304,20210


Balanced Accuracy Score: 0.7869403921690312

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.78      0.75     16936
           1       0.85      0.79      0.82     25514

    accuracy                           0.79     42450
   macro avg       0.78      0.79      0.78     42450
weighted avg       0.79      0.79      0.79     42450



## 5. Deep Neural Network

### Define a Deep Neural Network Model

In [36]:
# Define the basic neural network model
number_input_features = len(X_train_scaled[0]) # X_train.values[0]
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 90

nn = tf.keras.models.Sequential()

# First Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output Layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               2880      
                                                                 
 dense_1 (Dense)             (None, 90)                10890     
                                                                 
 dense_2 (Dense)             (None, 1)                 91        
                                                                 
Total params: 13,861
Trainable params: 13,861
Non-trainable params: 0
_________________________________________________________________


### Compile, Train, and Evaluate the Model

In [37]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

Epoch 1/50
3980/3980 [==============================] - 5s 1ms/step - loss: 0.4379 - accuracy: 0.7919
Epoch 2/50
3980/3980 [==============================] - 5s 1ms/step - loss: 0.4216 - accuracy: 0.8015
Epoch 3/50
3980/3980 [==============================] - 5s 1ms/step - loss: 0.4155 - accuracy: 0.8040
Epoch 4/50
3980/3980 [==============================] - 5s 1ms/step - loss: 0.4117 - accuracy: 0.8059
Epoch 5/50
3980/3980 [==============================] - 5s 1ms/step - loss: 0.4089 - accuracy: 0.8076
Epoch 6/50
3980/3980 [==============================] - 5s 1ms/step - loss: 0.4062 - accuracy: 0.8098
Epoch 7/50
3980/3980 [==============================] - 5s 1ms/step - loss: 0.4044 - accuracy: 0.8097
Epoch 8/50
3980/3980 [==============================] - 5s 1ms/step - loss: 0.4024 - accuracy: 0.8112
Epoch 9/50
3980/3980 [==============================] - 5s 1ms/step - loss: 0.4005 - accuracy: 0.8113
Epoch 10/50
3980/3980 [==============================] - 5s 1ms/step - loss: 0.399

In [38]:
# Print the results of the model
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.4302031397819519, Accuracy: 0.8048527836799622
